In [1]:
import os
import random
import re
import sys
import warnings

import numpy as np
import pandas as pd
from scipy.spatial.distance import squareform, pdist
from sklearn.cluster import KMeans,DBSCAN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import *
from tmap.netx.SAFE import SAFE_batch, get_SAFE_summary, SAFE_single
from tmap.tda import mapper, filter
from tmap.tda.metric import Metric
from tmap.tda.plot import vis_progressX, Color
from tmap.tda.utils import optimize_dbscan_eps, cover_ratio
from tmap.tda.cover import Cover
from tmap.tda.metric import Metric

warnings.filterwarnings("ignore")


In [2]:
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
import pandas as pd
import numpy as np
importr("vegan")

def envfit_metadata(genus_path,metadata_path):
    rcode = """
    genus_table <- read.csv('{path_data}',row.names = 1)
    metadata <- read.csv('{path_metadata}',row.names = 1)
    
    bray <- vegdist(genus_table)
    ord <- capscale(bray ~ -1)
    #plot(ord)
    """.format(path_data=genus_path,path_metadata=metadata_path)
    robjects.r(rcode)

    envfit_result = robjects.r(
    """
    fit <- envfit(ord,metadata,permutations = 999)
    fit$vectors
    """)
    fit_result = pd.DataFrame(columns=["r","pvals","Source","End"],index=robjects.r("colnames(metadata)"))
    fit_result.loc[:,"r"] = envfit_result[envfit_result.names.index("r")]
    fit_result.loc[:, "pvals"] = envfit_result[envfit_result.names.index("pvals")]
    fit_result.loc[:, ["Source","End"]] = np.array(envfit_result[envfit_result.names.index("arrows")])
    return fit_result


In [3]:

random.seed(100)
np.random.seed(100)

def get_projected_x(genus_tabs):
    tm = mapper.Mapper(verbose=0)
    dis = squareform(pdist(genus_tabs, metric="braycurtis"))
    metric = Metric(metric="precomputed")
    lens = [filter.PCOA(components=[0, 1], metric=metric, random_state=100)]
    projected_X = tm.filter(dis, lens=lens)
    projected_X = MinMaxScaler().fit_transform(projected_X)
    return projected_X

def generate_metadata_continuous(projected_X):
    p1, p2 = [random.uniform(-1, 1) for i in [1, 2]]
    metadata = p1*projected_X[:,0]+p2*projected_X[:,1]

    return metadata

In [4]:
def get_true_pred(graph,metadata,n_iter,threshold,raw=False):
    safe_scores = SAFE_batch(graph, meta_data=metadata, n_iter=n_iter, verbose=0)
    safe_summary = get_SAFE_summary(graph,metadata,safe_scores,n_iter,threshold)
    y_true = [1 if not _.endswith('fake') else 0 for _ in metadata.columns]
    y_pred = [1 if _!=0 else 0 for _ in safe_summary.loc[metadata.columns,'SAFE enriched score']]
    if raw:
        y_true = [1 if not _.endswith('fake') else 0 for _ in metadata.columns]
        y_pred = list(safe_summary.loc[metadata.columns,'SAFE enriched score'])
    return y_true,y_pred

In [5]:
otu_path = 'demo_mat.csv'
metadata_path = "metadata/metadata_linear.csv"
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import FloatVector
stats = importr('stats')

In [6]:
simulated_otu = pd.read_csv(otu_path, sep=',', index_col=0)
simulated_otu = simulated_otu.divide(simulated_otu.sum(1), axis=0)
dis = squareform(pdist(simulated_otu, metric="braycurtis"))

# generate graph
tm = mapper.Mapper(verbose=1)
metric = Metric(metric="precomputed")
lens = [filter.PCOA(components=[0, 1], metric=metric, random_state=100)]
projected_X = tm.filter(dis, lens=lens)
eps = optimize_dbscan_eps(simulated_otu, threshold=95)
clusterer = DBSCAN(eps=eps, min_samples=3)
r = 30
cover = Cover(projected_data=MinMaxScaler().fit_transform(projected_X), resolution=r, overlap=0.85)
graph = tm.map(data=simulated_otu, cover=cover, clusterer=clusterer)

Filtering by PCOA.
...calculate distance matrix using the precomputed metric.
Finish filtering of points cloud data.


 51%|█████     | 458/900 [00:00<00:00, 4577.88it/s]

Mapping on data (500, 300) using lens (500, 2)
...minimal number of points in hypercube to do clustering: 3


100%|██████████| 900/900 [00:00<00:00, 4081.62it/s]


...create 217 nodes.
...calculate projection coordinates of nodes.
...construct a TDA graph.
...create 1465 edges.
Finish TDA mapping


In [8]:
num_metadata = 50
def continuous(raw=False):
    projected_X = get_projected_x(simulated_otu)
    metadata = [generate_metadata_continuous(projected_X) for i in range(0, num_metadata)]
    metadata = pd.DataFrame(metadata).T
    metadata.columns = ['meta' + str(_) for _ in range(1, num_metadata + 1)]

    
    li_shuffled = metadata.copy().apply(lambda x: np.random.permutation(x),axis=0)
    li_shuffled.columns = [_ + '_fake' for _ in li_shuffled.columns]
    new_metadata = pd.concat([metadata, li_shuffled], axis=1)

    new_metadata.to_csv(metadata_path, index=True)
    y_true,y_pred = get_true_pred(graph,new_metadata,n_iter=1000,threshold=0.01,raw=raw)
    
    fit_result = envfit_metadata(otu_path, metadata_path)
    p_adjust = stats.p_adjust(FloatVector(fit_result.pvals.values), method = 'BH')
    fit_result["BH_corrected"] = p_adjust
    fit_y_true = y_true[::]
    if raw:
        fit_y_pred = list(fit_result.loc[new_metadata.columns,"pvals"])
    else:
        fit_y_pred = [1 if fit_result.loc[_,'BH_corrected']<=0.05 else 0 for _ in new_metadata.columns]
    return y_true,y_pred,fit_y_pred

In [9]:
from tqdm import trange

res = []
for i in trange(0,100):
    y_true,y_pred,fit_y_pred = continuous(raw=True)
    res.append((y_true,y_pred,fit_y_pred))

100%|██████████| 100/100 [42:12<00:00, 25.33s/it]


In [10]:
# basic rule
fsafe_recall = lambda x: recall_score(x[0],[1 if _ !=0 else 0 for _ in x[1]])
fsafe_precision = lambda x: precision_score(x[0],[1 if _!=0 else 0 for _ in x[1]])
fenvfit_recall = lambda x: recall_score(x[0],[1 if _<=0.05 else 0 for _ in x[2]])
fenvfit_precision = lambda x: precision_score(x[0],[1 if _<=0.05 else 0 for _ in x[2]])
fsafe_f1 = lambda x: f1_score(x[0],[1 if _ !=0 else 0 for _ in x[1]])
fenvfit_f1 = lambda x: f1_score(x[0],[1 if _<=0.05 else 0 for _ in x[2]])

In [11]:
# output raw y_true and y_pred
tmp = []
for idx,_ in enumerate(res):
    cache = pd.DataFrame(np.array(_).T,columns=['y_true_%s' % idx,
                                                'y_pred_%s' % idx,
                                                'y_envfit_%s' % idx])
    tmp.append(cache)
raw_result = pd.concat(tmp,axis=1)
raw_result.to_csv('plot_data/linear_100_raw.csv')